In [1]:
#import modules needed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy as sqla
from sqlalchemy import create_engine

In [2]:
URI="group-project.csuftpmigzgq.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbikes"
USER="alvaro"
PASSWORD="Oravla01"

In [3]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo = True)

In [4]:
sql1="""
Select description, dt, temp,temp_min, temp_max, humidity From dbikes.weather 
"""

In [5]:
sql2="""
Select id, number, available_bikes, available_bikes_stands, last_update From dbikes.availability 
"""

In [6]:
dfweather=pd.read_sql_query(sql1, engine)

2021-04-05 19:44:13,151 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-04-05 19:44:13,151 INFO sqlalchemy.engine.base.Engine ()
2021-04-05 19:44:13,261 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-04-05 19:44:13,261 INFO sqlalchemy.engine.base.Engine ()
2021-04-05 19:44:13,457 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-04-05 19:44:13,457 INFO sqlalchemy.engine.base.Engine ()
2021-04-05 19:44:13,663 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2021-04-05 19:44:13,663 INFO sqlalchemy.engine.base.Engine ()
2021-04-05 19:44:13,810 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-04-05 19:44:13,810 INFO sqlalchemy.engine.base.Engine ()
2021-04-05 19:44:13,976 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-04-05 19:44:13,976 INFO sqlalchemy.engine.base.E

In [7]:
dfweather.head()

,description,dt,temp,temp_min,temp_max,humidity
0,scattered clouds,2021-03-17 15:12:52,13,11,14,62
1,scattered clouds,2021-03-17 15:15:03,13,11,14,62
2,scattered clouds,2021-03-17 15:27:28,13,13,14,50
3,scattered clouds,2021-03-17 15:45:43,13,13,14,47
4,scattered clouds,2021-03-17 15:58:47,14,13,14,47


In [8]:
dfavailability=pd.read_sql_query(sql2,engine)

2021-04-05 19:44:24,272 INFO sqlalchemy.engine.base.OptionEngine 
Select id, number, available_bikes, available_bikes_stands, last_update From dbikes.availability 

2021-04-05 19:44:24,272 INFO sqlalchemy.engine.base.OptionEngine ()


In [9]:
dfavailability.shape

(662542, 5)

In [10]:
dfweather.head()

,description,dt,temp,temp_min,temp_max,humidity
0,scattered clouds,2021-03-17 15:12:52,13,11,14,62
1,scattered clouds,2021-03-17 15:15:03,13,11,14,62
2,scattered clouds,2021-03-17 15:27:28,13,13,14,50
3,scattered clouds,2021-03-17 15:45:43,13,13,14,47
4,scattered clouds,2021-03-17 15:58:47,14,13,14,47


In [11]:
dfavailability.head()

,id,number,available_bikes,available_bikes_stands,last_update
0,80,2,11,9,2021-02-25 19:16:01
1,516,2,11,9,2021-02-25 19:26:10
2,734,2,11,9,2021-02-25 19:36:18
3,952,2,11,9,2021-02-25 19:46:26
4,1170,2,11,9,2021-02-25 19:56:35


In [12]:
dfavailability.rename(columns = {'number':'StationNumber'}, inplace = True)

In [13]:
dfavailability.nunique()

id                        662542
StationNumber                110
available_bikes               41
available_bikes_stands        41
last_update               521515
dtype: int64

In [14]:
dfavailability.dtypes

id                                 int64
StationNumber                      int64
available_bikes                    int64
available_bikes_stands             int64
last_update               datetime64[ns]
dtype: object

In [15]:
dfavailability['last_update']=dfavailability['last_update'].apply(pd.to_datetime, format='%Y/%m/%d')

In [16]:
dfavailability.nunique()

id                        662542
StationNumber                110
available_bikes               41
available_bikes_stands        41
last_update               521515
dtype: int64

In [17]:
dfavailability.dtypes

id                                 int64
StationNumber                      int64
available_bikes                    int64
available_bikes_stands             int64
last_update               datetime64[ns]
dtype: object

In [18]:
dfweather.dtypes

description            object
dt             datetime64[ns]
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
dtype: object

In [19]:
dfweather.nunique()

description       9
dt             1839
temp             17
temp_min         19
temp_max         18
humidity         33
dtype: int64

In [20]:
dfweather['description'] = dfweather['description'].astype('category')

In [21]:
dfweather.dtypes

description          category
dt             datetime64[ns]
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
dtype: object

In [22]:
dfweather.isnull().sum().sort_values(ascending=False)/len(dfweather)*100

description    0.0
dt             0.0
temp           0.0
temp_min       0.0
temp_max       0.0
humidity       0.0
dtype: float64

In [23]:
dfavailability.isnull().sum().sort_values(ascending=False)/len(dfavailability)*100

id                        0.0
StationNumber             0.0
available_bikes           0.0
available_bikes_stands    0.0
last_update               0.0
dtype: float64

In [24]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', dfweather.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', dfweather[dfweather.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [25]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', dfavailability.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', dfavailability[dfavailability.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [26]:
dfavailability.describe().T

,count,mean,std,min,25%,50%,75%,max
id,662542.0,611408.115137,352408.913461,1.0,305679.25,613608.5,916191.75,1218723.0
StationNumber,662542.0,60.193728,35.076642,2.0,31.00,61.0,90.00,507.0
available_bikes,662542.0,10.707768,7.141264,0.0,5.00,10.0,15.00,40.0
available_bikes_stands,662542.0,21.272425,9.318119,0.0,15.00,21.0,28.00,40.0


In [28]:
dfavailability.head()

,id,StationNumber,available_bikes,available_bikes_stands,last_update
0,80,2,11,9,2021-02-25 19:16:01
1,516,2,11,9,2021-02-25 19:26:10
2,734,2,11,9,2021-02-25 19:36:18
3,952,2,11,9,2021-02-25 19:46:26
4,1170,2,11,9,2021-02-25 19:56:35


In [30]:
dfavailability['intervals']=dfavailability['last_update'].dt.round('15min')

In [31]:
dfavailability.head().sort_values('last_update')

,id,StationNumber,available_bikes,available_bikes_stands,last_update,intervals
0,80,2,11,9,2021-02-25 19:16:01,2021-02-25 19:15:00
1,516,2,11,9,2021-02-25 19:26:10,2021-02-25 19:30:00
2,734,2,11,9,2021-02-25 19:36:18,2021-02-25 19:30:00
3,952,2,11,9,2021-02-25 19:46:26,2021-02-25 19:45:00
4,1170,2,11,9,2021-02-25 19:56:35,2021-02-25 20:00:00


In [32]:
dfweather.describe().T

,count,mean,std,min,25%,50%,75%,max
temp,1839.0,8.887983,3.118351,1.0,7.0,9.0,11.0,17.0
temp_min,1839.0,7.977705,3.326787,-2.0,6.0,8.0,10.0,16.0
temp_max,1839.0,9.747145,2.937083,2.0,8.0,9.0,12.0,19.0
humidity,1839.0,79.158782,11.127602,42.0,71.0,81.0,87.0,100.0


In [33]:
dfweather.head()

,description,dt,temp,temp_min,temp_max,humidity
0,scattered clouds,2021-03-17 15:12:52,13,11,14,62
1,scattered clouds,2021-03-17 15:15:03,13,11,14,62
2,scattered clouds,2021-03-17 15:27:28,13,13,14,50
3,scattered clouds,2021-03-17 15:45:43,13,13,14,47
4,scattered clouds,2021-03-17 15:58:47,14,13,14,47


In [34]:
dfweather['intervals']=dfweather['dt'].dt.round('15min')

In [35]:
dfweather.head().sort_values('dt')

,description,dt,temp,temp_min,temp_max,humidity,intervals
0,scattered clouds,2021-03-17 15:12:52,13,11,14,62,2021-03-17 15:15:00
1,scattered clouds,2021-03-17 15:15:03,13,11,14,62,2021-03-17 15:15:00
2,scattered clouds,2021-03-17 15:27:28,13,13,14,50,2021-03-17 15:30:00
3,scattered clouds,2021-03-17 15:45:43,13,13,14,47,2021-03-17 15:45:00
4,scattered clouds,2021-03-17 15:58:47,14,13,14,47,2021-03-17 16:00:00


In [36]:
dfavailability.head().sort_values('last_update')

,id,StationNumber,available_bikes,available_bikes_stands,last_update,intervals
0,80,2,11,9,2021-02-25 19:16:01,2021-02-25 19:15:00
1,516,2,11,9,2021-02-25 19:26:10,2021-02-25 19:30:00
2,734,2,11,9,2021-02-25 19:36:18,2021-02-25 19:30:00
3,952,2,11,9,2021-02-25 19:46:26,2021-02-25 19:45:00
4,1170,2,11,9,2021-02-25 19:56:35,2021-02-25 20:00:00


#### Drop original DateTime columns as new interval columns will be used

In [38]:
mergeweather= dfweather.drop(['dt'], axis=1)

In [39]:
mergeavail=dfavailability.drop(['last_update'], axis=1)

In [40]:
mergeavail.dtypes

id                                 int64
StationNumber                      int64
available_bikes                    int64
available_bikes_stands             int64
intervals                 datetime64[ns]
dtype: object

In [41]:
mergeweather.dtypes

description          category
temp                    int64
temp_min                int64
temp_max                int64
humidity                int64
intervals      datetime64[ns]
dtype: object

In [42]:
mergeavail=mergeavail.sort_values('intervals')
mergeavail.isnull().sum()

id                        0
StationNumber             0
available_bikes           0
available_bikes_stands    0
intervals                 0
dtype: int64

In [51]:
mergeweather=mergeweather.sort_values('intervals')
mergeweather.isnull().sum()

description    0
temp           0
temp_min       0
temp_max       0
humidity       0
intervals      0
dtype: int64

In [52]:
MergedTables=pd.merge( mergeavail, mergeweather,how='inner', on='intervals')

In [53]:
MergedTables.dtypes

id                                 int64
StationNumber                      int64
available_bikes                    int64
available_bikes_stands             int64
intervals                 datetime64[ns]
description                     category
temp                               int64
temp_min                           int64
temp_max                           int64
humidity                           int64
dtype: object

In [54]:
MergedTables.isnull().sum().sort_values(ascending=False)/len(MergedTables)*100

id                        0.0
StationNumber             0.0
available_bikes           0.0
available_bikes_stands    0.0
intervals                 0.0
description               0.0
temp                      0.0
temp_min                  0.0
temp_max                  0.0
humidity                  0.0
dtype: float64

In [55]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', MergedTables.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', MergedTables[MergedTables.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  361
Number of duplicate rows (including first) in the table is: 722


In [56]:
MergedTables.shape

(328018, 10)

In [57]:
FinalMerge=MergedTables[~MergedTables.duplicated()]

In [59]:
FinalMerge.shape

(327657, 10)

In [60]:
FinalMerge['Day']=FinalMerge['intervals'].dt.day_name()

In [61]:
FinalMerge['Hour']=FinalMerge['intervals'].dt.hour

In [62]:
choices=['Morning', 'Afternoon', 'Evening']
conditions=[(FinalMerge['Hour'] > 6) & (FinalMerge['Hour'] < 12), (FinalMerge['Hour'] >= 12) & (FinalMerge['Hour'] < 16),(FinalMerge['Hour'] >= 16) & (FinalMerge['Hour'] < 23)]

In [63]:
FinalMerge['TimeOfDay']= np.select(conditions, choices, default='Night')

In [64]:
FinalMerge.head()

,id,StationNumber,available_bikes,available_bikes_stands,intervals,description,temp,temp_min,temp_max,humidity,Day,Hour,TimeOfDay
0,619550,115,20,10,2021-03-17 15:15:00,scattered clouds,13,11,14,62,Wednesday,15,Afternoon
2,619609,54,15,18,2021-03-17 15:15:00,scattered clouds,13,11,14,62,Wednesday,15,Afternoon
4,619620,98,4,36,2021-03-17 15:15:00,scattered clouds,13,11,14,62,Wednesday,15,Afternoon
6,619293,98,4,36,2021-03-17 15:15:00,scattered clouds,13,11,14,62,Wednesday,15,Afternoon
8,619282,54,15,18,2021-03-17 15:15:00,scattered clouds,13,11,14,62,Wednesday,15,Afternoon


## 2 Models will be used, one for Available bikes and one for availale bike stands

In [65]:
availableBikes=FinalMerge.drop(['intervals', 'Hour'], axis=1)
availableStands=FinalMerge.drop(['intervals', 'Hour'], axis=1)

In [67]:
availableBikes.to_csv('availablebikes.csv', index=False)
availableStands.to_csv('availablestands.csv', index=False)